In [ ]:
import numpy as np

from ase.spacegroup import crystal
from ase.visualize import view
from ase import Atom
# Visualisation with "ngl" viewer requires the nglview library
# see http://nglviewer.org/nglview/latest/index.html

### Create the unit cell

In [ ]:
a = 3.57
Ge3Mn5 = crystal(('Ge', 'Mn', 'Mn'), [(0.610,0.000,0.250),
                                      (0.250,0.000,0.250),
                                      (0.333333,0.666667,0.000)],
                 spacegroup=193,
                 cellpar=[7.184, 7.184, 5.053, 90, 90, 120]
                )

In [ ]:
view(Ge3Mn5, viewer='ngl')

### Add interstitial atoms in the unit cell

In [ ]:
z_pos = Ge3Mn5.cell.cellpar()[2] / 2
print('z_position:', z_pos)
interstitial_Ge_atom1 = Atom(symbol='Ge',
                             position=(0, 0, z_pos),
                             tag=1
                            )
interstitial_Ge_atom2 = Atom(symbol='Ge',
                             position=(0, 0, 0),
                             tag=2
                            )
Ge3Mn5.append(interstitial_Ge_atom1)
Ge3Mn5.append(interstitial_Ge_atom2)

In [ ]:
view(Ge3Mn5, viewer='ngl')

In [ ]:
Ge3Mn5.cell

### Make the cell orthorhombic

In [ ]:
from ase.build import make_supercell
# hexagonal to orthorhombric transformation
transformation_matrix = [[0, -1, 0],
                         [2, 1, 0],
                         [0, 0, 1]]
Ge3Mn5_orthorhombic = make_supercell(Ge3Mn5, transformation_matrix)

# Make a new Atoms object to reset the cell
from ase import Atoms
Ge3Mn5_orthorhombic = Atoms(symbols=Ge3Mn5_orthorhombic.get_chemical_symbols(),
                             scaled_positions=Ge3Mn5_orthorhombic.get_scaled_positions(),
                             cell=Ge3Mn5_orthorhombic.cell.cellpar(),
                             tags=Ge3Mn5_orthorhombic.get_array('tags'),
                             pbc=True
                            )

In [ ]:
Ge3Mn5_orthorhombic.cell

In [ ]:
view(Ge3Mn5_orthorhombic, viewer='ngl')

In [ ]:
def set_occupancy_interstitial(atoms, occupancy, label):
    """
    Set the occupancy to the atoms tag with "label".
    """
    atoms = atoms.copy()
    # Set default occupancy to 1:
    atoms.set_array('occupancies', np.ones_like(atoms.numbers, dtype='float'))

    # Set occupancy of Ge interstitial atom (label with the tag 1 or 2)
    for tag in label:
        atoms.arrays['occupancies'][np.where(atoms.get_array('tags') == tag)[0]] = occupancy

    return atoms

In [ ]:
occupancy = 0.1
atoms = set_occupancy_interstitial(Ge3Mn5_orthorhombic, occupancy, label=[1, 2])

### Export cell

In [ ]:
for occupancy in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    atoms = set_occupancy_interstitial(Ge3Mn5_orthorhombic, occupancy, [1, 2])
    atoms.write(f"Ge3Mn5_GeI_occ{occupancy}.xyz", format='prismatic',
                debye_waller_factors={'Ge':0.61, 'Mn':0.34})

In [ ]:
# Save for mustem
Ge3Mn5_orthorhombic.write("Ge3Mn5_GeI-single.xtl", format='mustem', keV=100,
                          debye_waller_factors={'Ge':0.61, 'Mn':0.4, 'C':0.05})

In [ ]:
# Save xyz for prismatic
Ge3Mn5_orthorhombic.write("Ge3Mn5_GeI-single.xyz", format='prismatic',
                          debye_waller_factors={'Ge':0.61, 'Mn':0.4})

### Visualise slicing

In [ ]:
from ase.geometry import get_layers
slice_thickness = 1.26325
atoms = Ge3Mn5_orthorhombic
indices = get_layers(atoms, (0,0,1), tolerance=0.25)[0]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms
fig, axes = plt.subplots(nrows=2, ncols=4, sharex=True, sharey='row', constrained_layout=True)
for layer_index in range(4):
    layer = atoms[indices==layer_index]
    #axes[0, layer_index].set_title(f'Slice #{layer_index}')
    plot_atoms(layer, axes[0, layer_index], radii=0.3)
    plot_atoms(layer, axes[1, layer_index], radii=0.3, rotation='90x')

plt.savefig('slicing.png', dpi=300)